In [4]:
import pandas as pd

In [5]:
prescriptions = pd.read_csv(r'../data_unzipped/PartD_13/PARTD_PRESCRIBER_PUF_NPI_DRUG_13.tab', delimiter='\t')
payments = pd.read_csv(r'payments_with_npi_13.csv')

/Users/zakscholl/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# find the number of prescription rows I have with payments that have NPIs
len(prescriptions[prescriptions.NPI.isin(payments.NPI)])

7899437

In [156]:
# TODO: get the drug names for each manufacturer with payments in this set
drugs = pd.read_csv(r'../drug_manufacturers.csv')

In [20]:
drugs[drugs.manufacturer_name.str.contains('Otsuka')]

,brand_name,generic_name,manufacturer_name
2706,BUSULFEX,BUSULFAN,"Otsuka America Pharmaceutical, Inc."
55970,SAMSCA,TOLVAPTAN,Otsuka America Pharmaceutical Inc.
72625,REXULTI,BREXPIPRAZOLE,"Otsuka America Pharmaceutical, Inc."
73697,PLETAL,CILOSTAZOL,"Otsuka America Pharmaceutical, Inc."
76569,Dacogen,DECITABINE,"Otsuka America Pharmaceutical, Inc."


In [11]:
payments['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].head()

0                  FOREST PHARMACEUTICALS, INC.
1                  FOREST PHARMACEUTICALS, INC.
2                  FOREST PHARMACEUTICALS, INC.
3           Otsuka America Pharmaceutical, Inc.
4    Boehringer Ingelheim Pharmaceuticals, Inc.
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, dtype: object

In [ ]:
# remove punctuation and case when merging the payments data set with drug manufacturers

# find number of drug manufacturers needed to cover 90% of the payments

# Forest Pharmaceuticals -> Forest Laboratories

In [26]:
payments['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].value_counts(
    normalize=True).sort_values(ascending=False)[:115].sum()

0.9001817236793079

Top 115 payment makers cover 90% of the payments

Goal is to make sure all of those are accurately merged with the manufacturers dataset

In [70]:
top_payers = payments['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].value_counts(
    normalize=True).sort_values(ascending=False)[:115]

In [71]:
sum(top_payers.index.isin(drugs.manufacturer_name))

43

So off the bat not many of the top payers match the manufacturers in the drug dataset

I'm betting it has to do with punctuation and case inconsistency, so I'll adjust that first to see if we can get better coverage

In [72]:
top_payers[:5]

FOREST PHARMACEUTICALS, INC.      0.054110
AstraZeneca Pharmaceuticals LP    0.051373
Novo Nordisk Inc                  0.047950
Pfizer Inc.                       0.043880
Janssen Pharmaceuticals, Inc      0.038103
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, dtype: float64

In [73]:
top_payers.index = top_payers.index.str.replace('[^\w\s]', '')
top_payers.index = top_payers.index.str.lower()

In [74]:
top_payers[:5]

forest pharmaceuticals inc        0.054110
astrazeneca pharmaceuticals lp    0.051373
novo nordisk inc                  0.047950
pfizer inc                        0.043880
janssen pharmaceuticals inc       0.038103
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, dtype: float64

In [157]:
drugs['cleaned'] = drugs.manufacturer_name.str.lower()

In [158]:
drugs['cleaned'] = drugs.cleaned.str.replace('[^\w\s]', '')

In [159]:
drugs.cleaned[:5]

0            native remedies llc
1            wh nutritionals llc
2                     five below
3    as medication solutions llc
4                    academy ltd
Name: cleaned, dtype: object

In [78]:
sum(top_payers.index.isin(drugs.cleaned))

62

In [103]:
sum(top_payers[top_payers.index.isin(drugs.cleaned)])

0.59215543958772099

So now we're at just over half in terms of number and percentage, which isn't so bad. But I'd like to see if we can easily bump that up

In [80]:
len(drugs[drugs.cleaned.isin(top_payers.index)])

1700

This encompasses about 1700 different prescription drugs. Probably not a bad sample that we can go ahead with, but I want to manually check a few of the ones that aren't matching...see if there is some commanality among them that we can overcome.

In [102]:
top_payers[~top_payers.index.isin(drugs.cleaned)][:10]

forest pharmaceuticals inc        0.054110
novo nordisk inc                  0.047950
lilly usa llc                     0.032739
merck sharp  dohme corporation    0.016893
horizon pharma usa inc            0.011651
genentech usa inc                 0.011625
warner chilcott llc               0.009874
boston scientific corporation     0.008685
shire us holdings                 0.008367
medtronic vascular inc            0.008092
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, dtype: float64

Well I know that the top case is because "Forest Pharmaceuticals" is the company name, but they manufacture under "Forest Laboratories"

In [95]:
drugs[drugs.cleaned.str.contains(r'novo nordisk')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
53413,Xultophy 100/3.6,(INSULIN DEGLUDEC AND LIRAGLUTIDE),Novo Nordisk,novo nordisk
55461,Vagifem,ESTRADIOL,Novo Nordisk,novo nordisk
62986,NovoLog,INSULIN ASPART,Novo Nordisk,novo nordisk
64388,Novolin 70/30,HUMAN INSULIN,Novo Nordisk,novo nordisk
65363,Norditropin,SOMATROPIN,Novo Nordisk,novo nordisk


In [91]:
drugs[drugs.cleaned.str.contains(r'lilly')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
37566,TALTZ,IXEKIZUMAB,Eli Lilly and Company,eli lilly and company
38573,Humulin 70/30,INSULIN HUMAN,Eli Lilly and Company,eli lilly and company
41500,Humulin N,INSULIN HUMAN,Eli Lilly and Company,eli lilly and company
41506,Portrazza,NECITUMUMAB,Eli Lilly and Company,eli lilly and company
52274,LARTRUVO,OLARATUMAB,Eli Lilly and Company,eli lilly and company


In [96]:
drugs[drugs.cleaned.str.contains(r'merck')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
2984,BRIDION,SUGAMMADEX,Merck Sharp & Dohme Corp.,merck sharp dohme corp
11734,Integrilin,EPTIFIBATIDE,Merck Sharp & Dohme Corp.,merck sharp dohme corp
36438,CUBICIN RF,DAPTOMYCIN,Merck Sharp & Dohme Corp.,merck sharp dohme corp
39398,CUBICIN RF,DAPTOMYCIN,Merck Sharp & Dohme Corp.,merck sharp dohme corp
43078,ZEPATIER,ELBASVIR AND GRAZOPREVIR,Merck Sharp & Dohme Corp.,merck sharp dohme corp


In [99]:
drugs[drugs.cleaned.str.contains(r'horizon pharma')]

,brand_name,generic_name,manufacturer_name,cleaned
62750,RAYOS,PREDNISONE,Horizon Pharma Inc.,horizon pharma inc
64109,Pennsaid,DICLOFENAC SODIUM,Horizon Pharma Inc.,horizon pharma inc
65127,Vimovo,NAPROXEN AND ESOMEPRAZOLE MAGNESIUM,Horizon Pharma Inc.,horizon pharma inc
65294,BUPHENYL,SODIUM PHENYLBUTYRATE,"Horizon Pharma, Inc.",horizon pharma inc
68026,DUEXIS,IBUPROFEN AND FAMOTIDINE,Horizon Pharma Inc.,horizon pharma inc


In [106]:
drugs[drugs.cleaned.str.contains(r'genentech')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
18289,TNKase,TENECTEPLASE,"Genentech, Inc.",genentech inc
39169,ALECENSA,ALECTINIB HYDROCHLORIDE,"Genentech, Inc.",genentech inc
39440,TECENTRIQ,ATEZOLIZUMAB,"Genentech, Inc.",genentech inc
46105,Esbriet,PIRFENIDONE,"Genentech, Inc.",genentech inc
54784,Xenical,ORLISTAT,"Genentech, Inc.",genentech inc


In [108]:
drugs[drugs.cleaned.str.contains(r'warner')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
6392,norethindrone acetate and ethinyl estradiol an...,"NORETHINDRONE ACETATE AND ETHINYL ESTRADIOL, A...","Warner Chilcott (US), LLC",warner chilcott us llc
8146,FEMCON Fe,"NORETHINDRONE AND ETHINYL ESTRADIOL, AND FERRO...","Warner Chilcott (US), LLC",warner chilcott us llc
12735,ESTROSTEP Fe,NORETHINDRONE ACETATE AND ETHINYL ESTRADIOL AN...,"Warner Chilcott (US), LLC",warner chilcott us llc
25591,Dovonex,CALCIPOTRIENE,"Warner Chilcott (US), LLC",warner chilcott us llc
25686,TACLONEX,CALCIPOTRIENE AND BETAMETHASONE DIPROPIONATE,"Warner Chilcott (US), LLC",warner chilcott us llc


In [112]:
drugs[drugs.cleaned.str.contains(r'boston')]

,brand_name,generic_name,manufacturer_name,cleaned


In [115]:
drugs[drugs.cleaned.str.contains(r'shire us')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
26955,ProAmatine,MIDODRINE HYDROCHLORIDE,Shire US Manufacturing Inc.,shire us manufacturing inc
32575,Xiidra,LIFITEGRAST,Shire US Manufacturing Inc.,shire us manufacturing inc
65855,Intuniv,GUANFACINE,Shire US Manufacturing Inc.,shire us manufacturing inc
67271,Fosrenol,LANTHANUM CARBONATE,Shire US Manufacturing Inc.,shire us manufacturing inc
69991,ELAPRASE,IDURSULFASE,Shire US Manufacturing Inc.,shire us manufacturing inc


In [118]:
drugs[drugs.cleaned.str.contains(r'medtronic')]

,brand_name,generic_name,manufacturer_name,cleaned
61366,Lioresal,BACLOFEN,Medtronic Neuromodulation,medtronic neuromodulation


In [120]:
drugs[drugs.cleaned.str.contains(r'forest lab')][:5]

,brand_name,generic_name,manufacturer_name,cleaned
44852,Thyrolar,"LIOTHYRONINE SODIUM, LEVOTHYROXINE SODIUM","Forest Laboratories, Inc",forest laboratories inc
59982,Armour Thyroid,"THYROID, PORCINE","Forest Laboratories, Inc",forest laboratories inc
62709,Namenda,MEMANTINE HYDROCHLORIDE,"Forest Laboratories, Inc.",forest laboratories inc
70065,Daliresp,ROFLUMILAST,"Forest Laboratories, Inc.",forest laboratories inc
71064,Namenda XR,MEMANTINE HYDROCHLORIDE,"Forest Laboratories, Inc.",forest laboratories inc


So save for boston scientific and medtronic vascular, with a few modifications to the names that these are associated with, we can pick up about another 12% coverage. Well worth it for a little bit of manual digging.

To merge these datasets we need to modify the manufacturer names broadly to be all lowercase, without punctuation, and then add some special cases for these eight manufacturers. 

Once we've done that we can start looking at the drug names that those manufacturers manufacture and look at the coverage on our prescription dataset.

So let's take the drugs dataset and make those modifications to the 'cleaned' column and save it so we can merge it in future work.

In [160]:
# specific changes based on the top 8 
drugs['cleaned'] = drugs['cleaned'].str.replace('novo nordisk', 'novo nordisk inc')
drugs['cleaned'] = drugs['cleaned'].str.replace('forest laboratories', 'forest pharmaceuticals inc')
drugs['cleaned'] = drugs['cleaned'].str.replace('forest laboratories inc', 'forest pharmaceuticals inc')
drugs['cleaned'] = drugs['cleaned'].str.replace('eli lilly and company', 'lilly usa llc')
drugs['cleaned'] = drugs['cleaned'].str.replace('merck sharp  dohme corp', 'merck sharp  dohme corporation')
drugs['cleaned'] = drugs['cleaned'].str.replace('horizon pharma inc', 'horizon pharma usa inc')
drugs['cleaned'] = drugs['cleaned'].str.replace('genentech inc', 'genentech usa inc')
drugs['cleaned'] = drugs['cleaned'].str.replace('warner chilcott us llc', 'warner chilcott llc')
drugs['cleaned'] = drugs['cleaned'].str.replace('shire us manufacturing inc', 'shire us holdings')

In [173]:
 sum(top_payers.index.isin(drugs.cleaned))

70

In [174]:
sum(top_payers[top_payers.index.isin(drugs.cleaned)])

0.78536349770831404

So perfect, we picked up another 6 matches and upped our coverage over 75% of the top 90% of payment companies.

In [175]:
top_payers[~top_payers.index.isin(drugs.cleaned)][:10]

boston scientific corporation    0.008685
medtronic vascular inc           0.008092
questcor pharmaceuticals         0.005750
stryker corporation              0.005480
st jude medical inc              0.005066
medtronic usa inc                0.004906
intuitive surgical inc           0.004305
salix pharmaceuticals ltd        0.004280
abbott laboratories              0.003470
depuy synthes sales inc          0.003213
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, dtype: float64

In [177]:
%ls ../

2_5_work/
Data Story.ipynb
Drug brand and manufacturer table creation.ipynb
Payments Exploration.ipynb
Payments Parsing.ipynb
Physician Profile ID -> NPID.ipynb
Prescription Analysis.ipynb
data/
data.zip
data_unzipped/
drug_manufacturers.csv


So let's save this dataset so we can use it in future work.

In [178]:
#drugs.to_csv(r'../drug_manufacturers_cleaned.csv', index=False)

In [179]:
payments['manufacturer_cleaned'] = payments['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].str.replace('[^\w\s]', '')
payments['manufacturer_cleaned'] = payments['manufacturer_cleaned'].str.lower()

And now we can save the payments data set which has the NPI data and a cleaned manufacturer column that we can use to join with the drug manufacturer dataset

In [181]:
#payments.to_csv(r'payments_with_npi_cleaned_13.csv', index=False)